In [1]:
# from load_csv import CSV_Loader
from configs.configuration import dataset_config
import pandas as pd
import glob
import dask.dataframe as dd

In [2]:
dataset_config

{'REFIT_DATA_FOLDER': '/data/external/refit/', 'REFIT_DATA_TYPE': '.csv'}

In [103]:

class _Loader:
    """
    Interface that loads all the data into the memory
    """

    def __init__(self):
        try:
            pass
            
        except Exception as e:
            display("Error occured in initialization of _Loader interface due to ", e)
                
        finally:
            display("Loader Interface initialized")
        
    @staticmethod
    def _load_file():
        raise NotImplementedError    


class CSV_Loader(_Loader):
    """
    
    """
    def __init__(self):
        try:
            super().__init__()
        
        except Exception as e:
            display("Error occured in initialization of CSV_Loader class due to ", e)
                
        finally:
            display("CSV_Loader initialized")
        
    @staticmethod
    def _load_file(csv_file_path,
                   index_column_name=None,
                   _nrows=None,
                   _iterator=True,
                   _chunksize=100000):
        try:
            tp = pd.read_csv(csv_file_path, nrows=_nrows, index_col=index_column_name, iterator=_iterator, chunksize=_chunksize) ## loading data in chunks reduces 90 percent execution time 
            df = pd.concat(tp, ignore_index=False)
            df.info(verbose=False, memory_usage="deep")
            return df  
        
        except Exception as e:
            display("Error occured in _load_file method of CSV_Loader class due to ", e)
    
    @staticmethod
    def _load_file_via_dask(csv_file_path,
                            specifications):
        try:
            display(f"Loading specified houses: {data_dict['fetch_houses']}")
            files = [f'{csv_file_path}House_{i}.csv' for i in data_dict['fetch_houses']]
            return [dd.read_csv(file) for file in files]
        
        except Exception as e:
            display("Error occured in _load_file_via_dask method of CSV_Loader class due to ", e)
            
            
    ###### appliance wise Dict Reader


In [104]:
data_dict = {
    'fetch_houses' : [1]
#     'appliance': 'kettle'
}

In [105]:
ob = CSV_Loader()

'Loader Interface initialized'

'CSV_Loader initialized'

In [106]:
%%time
f = ob._load_file_via_dask(csv_file_path=dataset_config['REFIT_DATA_FOLDER'], specifications=data_dict)

'Loading specified houses: [1]'

Wall time: 18 ms


In [107]:
%%time
d = f[0].compute().set_index('Time')
d = f[0].compute().set_index('Time')
d = f[0].compute().set_index('Time')

Wall time: 12.1 s


In [88]:
d

,Unix,Aggregate,Appliance1,Appliance2,Appliance3,Appliance4,Appliance5,Appliance6,Appliance7,Appliance8,Appliance9
Time,,,,,,,,,,,
2013-10-09 13:06:17,1381323977,523,74,0,69,0,0,0,0,0,1
2013-10-09 13:06:31,1381323991,526,75,0,69,0,0,0,0,0,1
2013-10-09 13:06:46,1381324006,540,74,0,68,0,0,0,0,0,1
2013-10-09 13:07:01,1381324021,532,74,0,68,0,0,0,0,0,1
2013-10-09 13:07:15,1381324035,540,74,0,69,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2015-07-10 11:56:05,1436529365,187,0,45,0,0,0,0,0,0,1
2015-07-10 11:56:12,1436529372,185,0,45,0,0,0,0,0,0,1
2015-07-10 11:56:18,1436529378,181,0,45,0,0,0,0,0,0,1


In [ ]:
%%time
ob._load_file(csv_file_path=dataset_config['REFIT_DATA_PATH'], index_column_name='Time')

In [ ]:


files = glob.glob(dataset_config['REFIT_DATA_FOLDER'] + '*' + dataset_config['REFIT_DATA_TYPE'])
print(files)
# data = [pd.read_csv(file, index_col='Time') for file in files]
# data
files[0]

In [ ]:
%%time



In [ ]:
data

In [ ]:
%%time
data = pd.read_csv(files[0], index_col='Time')

In [ ]:

import csv


In [ ]:
files

In [ ]:
%%time
data1 = dask.dataframe.read_csv(files[0])
data2 = dask.dataframe.read_csv(files[1])
data3 = dask.dataframe.read_csv(files[2])
data4 = dask.dataframe.read_csv(files[3])

df1=data1.compute()
df2=data2.compute()
df3=data3.compute()
df4=data4.compute()

In [ ]:
df1

In [ ]:
df

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, file='d://sample.log')

def hypotenuse(a, b):
    """Compute the hypotenuse"""
    return (a**2 + b**2)**0.5

kwargs = {'a':3, 'b':4, 'c':hypotenuse(3, 4)}

logging.debug("a = {a}, b = {b}".format(**kwargs))
logging.info("Hypotenuse of {a}, {b} is {c}".format(**kwargs))
logging.warning("a={a} and b={b} are equal".format(**kwargs))
logging.error("a={a} and b={b} cannot be negative".format(**kwargs))
logging.critical("Hypotenuse of {a}, {b} is {c}".format(**kwargs))

In [ ]:
import parser
print("Program to demonstrate parser module in Python")
print("\n")
exp = "5 + 8"
print("The given expression for parsing is as follows:")
print(exp)
print("\n")
print("Parsing of given expression results as: ")
st = parser.expr(exp)
print(st)